# Setup

In [1]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=09f3fc9cad6ec30a6fc0de6cc50006ac713e4c2169039e58f7f029cc4fc9de00
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pickle
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from keras.preprocessing.text import Tokenizer

# Deploy do modelo

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
modeloTransformersImportado = ('/content/drive/MyDrive/Modulo_6_grupo_1/modelo_rede_neural_Transformers.pkl')

In [5]:
with open(modeloTransformersImportado, 'rb') as f:
    modeloTransformers = pickle.load(f)

In [6]:
vetorizador = SentenceTransformer('distiluse-base-multilingual-cased')

# Teste do modelo importado

In [7]:
dadosteste = {
    'frase': ["esse o pior banco que ja vi em toda minha vida","muito ruim", "muito bom", "adorei", "nao gostei muito nao", "."],
}
dfteste = pd.DataFrame(dadosteste)
dfteste

,frase
0,esse o pior banco que ja vi em toda minha vida
1,muito ruim
2,muito bom
3,adorei
4,nao gostei muito nao
5,.


In [8]:
def prever1(dados):
  dados = vetorizador.encode(dados.tolist())
  dados = pd.DataFrame(dados)
  previsoes = modeloTransformers.predict(dados)
  previsoes = np.argmax(previsoes, axis=1)
  return previsoes

In [9]:
prever1(dfteste["frase"])

1/1 [==============================] - 0s 232ms/step


array([2, 2, 0, 0, 0, 1])

In [10]:
dfteste['sentimento'] = pd.Series(prever1(dfteste["frase"]))

1/1 [==============================] - 0s 23ms/step


In [11]:
dfteste

,frase,sentimento
0,esse o pior banco que ja vi em toda minha vida,2
1,muito ruim,2
2,muito bom,0
3,adorei,0
4,nao gostei muito nao,0
5,.,1


In [12]:
# df_ = dfteste
# df_['sentimento'] = pd.Series(prever1(df_["frase"]))
# df_['sentimento'] = df_['sentimento'].replace({0: 'POSITIVE',1: 'NEUTRAL', 2: 'NEGATIVE'})
# df_['texto'] = df_['frase'] + ' [' + df_['sentimento'] + ']'


# quantidades = df_['sentimento'].value_counts()
# labels = quantidades.index
# fig, ax = plt.subplots()
# ax.pie(quantidades, labels=labels, autopct='%1.1f%%')
# ax.set_title('Sentimentos do corpus')

In [13]:
# df_


In [14]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 k

In [15]:
import gradio as gr

In [16]:
import matplotlib.pyplot as plt
import numpy as np

# Pipeline

In [17]:
!pip install unidecode
import pandas as pd
from unidecode import unidecode
import re
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
!pip install -U spacy
!python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


2023-06-15 20:32:07.941008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 57.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [18]:
#Código para remoção dos acentos
def remocaoAcentos(dataframe):
    dataframe = dataframe.applymap(lambda x: unidecode(str(x)))
    return dataframe

In [19]:
def tratamentoMaiusculas(dataframe):
    # Pré-processamento das letras maiúsculas
    dataframe = dataframe.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # Exibição dos dados pré-processados
    return dataframe

In [20]:
def tratamentoAbreviacoes(coluna):
    coluna = coluna.astype(str)
    coluna = coluna.str.lower()

    # Dicionário com as abreviações e suas expansões, incluindo as palavras a serem substituídas
    abreviacoes = {
        r'\bvc\b': 'você',
        r'\btbm\b': 'também',
        r'\bpq\b': 'porque',
        r'\bmt\b': 'muito',
        r'\bmto\b': 'muito',
        r'\bblz\b': 'beleza',
        r'\bjah\b': 'já',
        r'\bbj\b': 'beijo',
        r'\bflw\b': 'falou',
        r'\bvlw\b': 'valeu',
        r'\bkrl\b': 'caralho',
        r'\bqnd\b': 'quando',
        r'\bfds\b': 'fim de semana',
        r'\bqq\b': 'qualquer',
        r'\bmsg\b': 'mensagem',
        r'\bpra\b': 'para',
        r'\bp\b': 'para',
        r'\bpqp\b': 'puta que pariu',
        r'\bvlh\b': 'velho',
        r'\bnss\b': 'nossa',
        r'\bngm\b': 'ninguém',
        r'\bqm\b': 'quem',
        r'\bobs\b': 'observação',
        r'\bqt\b': 'quanto',
        r'\btbm ñ\b': 'também não',
        r'\bpf\b': 'por favor',
        r'\bpdc\b': 'pode crer',
        r'\btalkei\b': 'ok',
        r'\bd+\b': 'demais',
        r'\bdms\b': 'demais',
        r'\bsqn\b': 'só que não',
        r'\bmsm\b': 'mesmo',
        r'\bn/ao\b': 'não',
        r'\bn/\b': 'não',
        r'\bblz\b': 'beleza',
        r'\bq\b': 'que',
        r'\btd\b': 'tudo',
        r'\bpfvr\b': 'por favor',
        r'\bgnt\b': 'gente',
        r'\bto\b': 'estou',
        r'\bvcs\b': 'vocês',
        r'\bn/oa\b': 'não',
        r'\bsla\b': 'sei lá',
        r'\bflws\b': 'falou',
        r'\bbjos\b': 'beijos',
        r'\bkd\b': 'cadê',
        r'\blgl\b': 'legal',
        r'\bnum\b': 'número',
        r'\bjá q\b': 'já que',
        r'\bmano\b': 'irmão',
        r'\bpke\b': 'porque',
        r'\bpr\b': 'para',
        r'\btpm\b': 'tensão pré-menstrual',
        r'\btá\b': 'está',
        r'\bqmto\b': 'muito',
        r'\bñ\b': 'não',
        r'\bvamu\b': 'vamos',
        r'\btmj\b': 'estamos juntos',
        r'[0-9]': '',
        r'@\w+': '',
        r'[^\w\s]': '',
        r'\bwww\.[^\s]*': '',
        r'\bola\b': '',
        r'\n': '',
        r'\bbtg\b': '',
        r'\bpactual\b': '',
        r'\bvocê\b': '',
        r'\bdia\b': '',
        r'\bjá\b': '',
        r'\bsão\b': '',
        r'\bsó\b': '',
        r'\blink\b': '',
        r'\br\b': '',
        r'\brepost\b': '',
        r'\baté\b': '',
        r'\bpaulo\b': '',
        r'\bbio\b': '',
        r'\bdia\b': '',
        r'\bdias\b': '',
        r'\bsera\b': '',
        r'\blá\b': '',
        r'\bmês\b': '',
        r'\bestão\b': '',
        r'\balém\b': '',
        r'\bano\b': '',
        r'\banos\b': '',
    }

    # Aplica a substituição de cada abreviação no dataframe
    for abreviacao, expansao in abreviacoes.items():
        coluna = coluna.apply(lambda x: re.sub(abreviacao, expansao, x, flags=re.IGNORECASE))

    return coluna

In [21]:
def remocaoStopWords(coluna):
    listaSemStopWords = []
    for texto in coluna:
        # converte o texto em um documento do Spacy
        doc = nlp(texto)
        # filtra os tokens que não são stop words
        tokenSemStopwords = [token.text for token in doc if not token.is_stop]
        # adiciona a lista de frases sem stop words
        listaSemStopWords.append(' '.join(tokenSemStopwords))
    # substitui a coluna original pelos textos sem stop words
    coluna[:] = listaSemStopWords
    return coluna

In [22]:
def pipeline(dados):
  #cria um novo df a ser tratado
  dadosT =  pd.DataFrame({'dadosTratados': dados})
  # substituição das letras maiusculas dos dados
  dadosT['dadosTratados'] = tratamentoMaiusculas(dadosT)
  #tratamento de stopwords
  dadosT['dadosTratados'] = remocaoStopWords(dadosT['dadosTratados'])
  #tratamento de abreviações
  dadosT['dadosTratados'] = tratamentoAbreviacoes(dadosT['dadosTratados'])
  # remove os acentos dos dados
  dadosT['dadosTratados'] = remocaoAcentos(dadosT)
  ## tratamento dos emojis
  #dadosT = conversaoEmojis(dadosT)
  ## lemmatização
  #dadosT['dadosTratados'] = textoLemmatizado(dadosT['dadosTratados'])
  return dadosT['dadosTratados']

In [23]:
dadosTratados = pipeline(dfteste["frase"])
dadosTratados

0    pior banco ja vi vida
1                     ruim
2                         
3                   adorei
4           nao gostei nao
5                         
Name: dadosTratados, dtype: object

# Contagem de palavras

In [24]:

def tokenizacaoPalavra(comentarios):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(comentarios)
  return tokenizer

In [25]:
tokenizacaoPalavra(dfteste["frase"])

In [26]:
def contarPalavrasFrequentes(tokenizer, top_n):
    contagem_palavras = tokenizer.word_counts
    palavras_frequentes = dict(sorted(contagem_palavras.items(), key=lambda item: item[1], reverse=True)[:top_n])
    return palavras_frequentes


Teste

In [27]:
tokenizer = tokenizacaoPalavra(dfteste["frase"])
palavras_frequentes = contarPalavrasFrequentes(tokenizer, 15)
print(palavras_frequentes)

{'muito': 3, 'nao': 2, 'esse': 1, 'o': 1, 'pior': 1, 'banco': 1, 'que': 1, 'ja': 1, 'vi': 1, 'em': 1, 'toda': 1, 'minha': 1, 'vida': 1, 'ruim': 1, 'bom': 1}


In [28]:
def frequentes(dados):
  dic = tokenizacaoPalavra(dados)
  palavras_frequentes = contarPalavrasFrequentes(dic, 15)

  return palavras_frequentes

Teste

In [29]:
frequentes(dfteste["frase"])

{'muito': 3,
 'nao': 2,
 'esse': 1,
 'o': 1,
 'pior': 1,
 'banco': 1,
 'que': 1,
 'ja': 1,
 'vi': 1,
 'em': 1,
 'toda': 1,
 'minha': 1,
 'vida': 1,
 'ruim': 1,
 'bom': 1}

## Grafico com palavras mais frequentes

In [30]:
def gerarGraficoFrequencia(frequencia):
  # Cria um DataFrame a partir do dicionário de frequência
  df_frequencia = pd.DataFrame.from_dict(frequencia, orient='index', columns=['Contagem'])

  # Ordena o DataFrame pela contagem em ordem decrescente
  df_frequencia = pd.DataFrame.from_dict(frequencia, orient='index', columns=['Contagem'])

  # Ordena o DataFrame pela contagem em ordem decrescente
  df_frequencia = df_frequencia.sort_values('Contagem', ascending=False)

  # Cria o gráfico de barras
  plt.figure(figsize=(10, 6))
  plt.bar(df_frequencia.index, df_frequencia['Contagem'])
  plt.xlabel('Palavra')
  plt.ylabel('Contagem')
  plt.title('Frequência das Palavras')
  plt.xticks(rotation=45)
  plt.tight_layout()
  # Armazena o gráfico de barras na variável graficofrequencia
  graficofrequencia = plt.gcf()
  return graficofrequencia

# sentimentos frequentes

In [31]:
def sentimentoFrequentes(dados):
  sentimentoMaisFrequentes = frequentes(dados)
  sentimentoMaisFrequentes = list(sentimentoMaisFrequentes.keys())
  sentimentoMaisFrequentes = pd.DataFrame({'palavra': sentimentoMaisFrequentes})
  sentimentoM = prever1(sentimentoMaisFrequentes['palavra'])
  sentimentoMaisFrequentes['previsto'] = sentimentoM
  return sentimentoMaisFrequentes

In [32]:
sentimentoFrequentes(dfteste["frase"])

1/1 [==============================] - 0s 24ms/step


,palavra,previsto
0,muito,0
1,nao,1
2,esse,1
3,o,1
4,pior,1
5,banco,1
6,que,1
7,ja,1
8,vi,1
9,em,1


# Nuvem de palavras

In [47]:
from wordcloud import WordCloud
import tempfile


In [50]:
def gerarNuvemPalavras(dicionarioFreq):
    # Cria uma instância da classe WordCloud com as configurações desejadas
    nuvemPalavras = WordCloud(width=8000, height=4000, background_color='white', colormap='viridis')

    # Gera a nuvem de palavras a partir do dicionário de frequências
    nuvemPalavras.generate_from_frequencies(dicionarioFreq)

    # Exibe a nuvem de palavras
    plt.figure(figsize=(12,6))
    plt.imshow(nuvemPalavras, interpolation='bilinear')
    plt.axis('off')
    plt.show()
    nuvem = plt.gcf()


Teste

In [51]:
gerarNuvemPalavras(frequentes(dfteste["frase"]))

'/tmp/tmpkpici8jp.png'

# Gradio

In [52]:
def label(file, nome_da_coluna_texto):
    df = pd.read_csv(file.name)

    df['sentimento'] = pd.Series(prever1(df[nome_da_coluna_texto]))
    df['sentimento'] = df['sentimento'].replace({0: 'POSITIVE',1: 'NEUTRAL', 2: 'NEGATIVE'})

    #lista dos textos
    df['Primeiras 5 frases'] = df[nome_da_coluna_texto] + '  -  [' + df['sentimento'] + ']'
    texto = pd.DataFrame(df['Primeiras 5 frases']).head()

    #frequencia
    frequencia = frequentes(pipeline(df[nome_da_coluna_texto]))

    # Armazena o gráfico de barras na variável graficofrequencia
    graficofrequencia = gerarGraficoFrequencia(frequencia)

    # Cria uma lista com os sentimentos frequentes
    sentimentosFrequentes = sentimentoFrequentes(df[nome_da_coluna_texto])
    sentimentosFrequentes = sentimentosFrequentes.replace({0: 'POSITIVE',1: 'NEUTRAL', 2: 'NEGATIVE'})
    sentimentosFrequentes = sentimentosFrequentes.to_string(index=False)


    # Cria uma nuvem de palavras
    nuvem = gerarNuvemPalavras(frequencia)

    #criacao do grafico
    quantidades = df['sentimento'].value_counts()
    labels = quantidades.index
    fig, ax = plt.subplots()
    fig.patch.set_facecolor('#1F2937') #mudar a cor de fundo do grafico

    ax.pie(quantidades, labels=labels, autopct='%1.1f%%')
    ax.set_title('Sentimentos do corpus', color='white')

    x = str(quantidades)

    #tamanho do corpus do corpus
    num_tuplas = "Número de Tuplas: " + str(len(df))

    return [num_tuplas,x,graficofrequencia, fig, sentimentosFrequentes , nuvem,  texto.to_html()]


iface = gr.Interface(fn=label,
                     inputs=["file", "text"],
                     outputs=["text", "text", "plot","plot","text","image", gr.outputs.HTML()],
                     title= "Analize de Sentimento")
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9ecf1d355bce3ded22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
